In [41]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install --upgrade tensorflow



# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()
sc._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")


Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (2.2.0)


In [0]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-05-15 20:42:56--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.04MB/s    in 0.8s    

2020-05-15 20:42:58 (1.04 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [27]:
from pyspark import SparkFiles

url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Sports_v1_00.tsv.gz"), sep="\t", header=True)
df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

In [29]:
count = df.count()
count

4850360

In [30]:
df.columns

['marketplace',
 'customer_id',
 'review_id',
 'product_id',
 'product_parent',
 'product_title',
 'product_category',
 'star_rating',
 'helpful_votes',
 'total_votes',
 'vine',
 'verified_purchase',
 'review_headline',
 'review_body',
 'review_date']

In [38]:
#create new tables to match the schemas
review_id_df = df[["review_id", "customer_id", "product_id", "product_parent", "review_date"]]

review_id_df.dtypes

[('review_id', 'string'),
 ('customer_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'string'),
 ('review_date', 'string')]

In [39]:
products_df = df[["product_id", "product_title"]]
products_df.dtypes

[('product_id', 'string'), ('product_title', 'string')]

In [52]:
customers_df = df[['customer_id']]
customers_df.show()

+-----------+
|customer_id|
+-----------+
|   48945260|
|    5782091|
|   45813853|
|    1593730|
|   29605511|
|   11112959|
|     108031|
|   13981540|
|   37993909|
|   26040213|
|   34657602|
|   14346192|
|   38782687|
|   27138575|
|   11838771|
|     535800|
|   23156579|
|   48107879|
|   27260960|
|   39537314|
+-----------+
only showing top 20 rows



In [49]:
#convert columns to match schemas
#from pyspark.sql.functions import regexp_extract, length
review_id_df

from pyspark.sql.types import IntegerType
review_id_df = review_id_df.withColumn("customer_id", review_id_df["customer_id"].cast(IntegerType()))
review_id_df = review_id_df.withColumn("product_parent", review_id_df["product_parent"].cast(IntegerType()))

review_id_df.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'string')]

In [0]:
mode = "append"
jdbc_url = "hwpostgresdb.c0kxkpm187ak.us-east-2.rds.amazonaws.com:5431/bigdata_hw_db"
config = {
    "user": "postgres",
    "password": "xxxxxxx",
    "driver": "org.postgresql.Driver"
}


In [54]:
review_id_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)


Py4JJavaError: ignored

In [0]:
products_df.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)
